## Máster en Big Data y Data Science

### TFM

#### MODELOS DE APRENDIZAJE NO SUPERVISADO : CLUSTERING

---

En esta libreta se generan los modelos de clusterización sobre el dataset final del escenario para su posterior evaluación y análisis. Se utilizará como herramienta de soporte a mlflow para el registro completo de la experimentación. 

---

In [87]:
!%pip install pandas

zsh:fg:1: no job control in this shell.


In [88]:
!%pip install kmodes

zsh:fg:1: no job control in this shell.


In [2]:
!%pip install mlflow

zsh:fg:1: no job control in this shell.


In [3]:
# Importación de librerías

import pandas as pd

# Se importa mlflow para registro de la experimentación 
import mlflow

# Se importan los métodos a utilizar para clusterizar
from kmodes.kmodes import KModes

# Se importan las métricas a utilizar para evaluar el proceso
from sklearn.metrics import davies_bouldin_score, homogeneity_score, completeness_score, v_measure_score

from sklearn.preprocessing import OneHotEncoder

----

##### Lectura del dataset

In [4]:
df = pd.read_csv('/Users/lorenadlmd/Desktop/Master BIGDATA/BIGDATA/TFM/python_VisualStudio/modelado/datos/dataset_transformado_final.csv', sep=",")
df.head(5)

,Unnamed: 0,Tipo_producto,Condición,Localidad,Marca,precio,precio_envio
0,1,auriculares,totalmente nuevo,Desconocido,otra,barato,desconocido
1,2,auriculares,totalmente nuevo,Desconocido,lenovo,barato,desconocido
2,3,auriculares,totalmente nuevo,Desconocido,xiaomi,muy barato,gratis
3,4,auriculares,totalmente nuevo,china,jbl,barato,gratis
4,6,auriculares,totalmente nuevo,Desconocido,otra,muy barato,gratis


In [5]:
df = df.applymap(lambda s: s.lower() if type(s) == str else s)
df.head()

/var/folders/rs/9vrff58j4zdcj52xz4pt7w3r0000gn/T/ipykernel_38472/1064296010.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda s: s.lower() if type(s) == str else s)


,Unnamed: 0,Tipo_producto,Condición,Localidad,Marca,precio,precio_envio
0,1,auriculares,totalmente nuevo,desconocido,otra,barato,desconocido
1,2,auriculares,totalmente nuevo,desconocido,lenovo,barato,desconocido
2,3,auriculares,totalmente nuevo,desconocido,xiaomi,muy barato,gratis
3,4,auriculares,totalmente nuevo,china,jbl,barato,gratis
4,6,auriculares,totalmente nuevo,desconocido,otra,muy barato,gratis


In [6]:
mlflow.end_run()

----

##### Configuración de la experimentación

Experimentacon con kmodes para distintas configuraciones

In [8]:
# Se genera el experimento en mlflow
exp_name = 'Experimentación clustering1: kmodes'
exp_id = mlflow.create_experiment(name=exp_name)

In [10]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import pairwise_distances

In [13]:
# Definir los parámetros a evaluar
k_values = [2, 3]
init_methods = ['Huang', 'Cao']

for k in k_values:
    for init in init_methods:
        with mlflow.start_run():
            # Ajustar el modelo K-Modes
            km = KModes(n_clusters=k, init=init, n_init=5)
            labels = km.fit_predict(df)

            # Calcular métricas (ejemplo: Davies-Bouldin Index)

            # Necesidad de convertir a numericas las variables para el calculo de DBS: one hot encoding
            encoder = OneHotEncoder(sparse_output=False)
            X_encoded = encoder.fit_transform(df)

            #Calcular el Davies-Bouldin-Score
            davies_bouldin = davies_bouldin_score(X_encoded, labels)

            # Calcular las distancias entre todos los puntos utilizando distancia Hamming
            distancias = pairwise_distances( X_encoded, metric='hamming')

            # Calcular el Silhouette Score
            silhouette_avg = silhouette_score(distancias, labels, metric="precomputed")

            # Registrar parámetros y métricas en MLflow
            mlflow.log_param("k", k)
            mlflow.log_param("init_method", init)
            mlflow.log_metric("davies_bouldin_score", davies_bouldin )
            mlflow.log_metric("silhouette_score", silhouette_avg )

            # registrar el modelo si lo deseas
            #mlflow.log_artifact("path/to/your/model")  # Si guardas el modelo

print("Experimentos registrados en MLflow.")

Experimentos registrados en MLflow.


In [14]:
# Visualizar resultados en mlflow
!mlflow ui

[2024-11-07 14:03:38 +0100] [43966] [INFO] Starting gunicorn 23.0.0
[2024-11-07 14:03:38 +0100] [43966] [INFO] Listening at: http://127.0.0.1:5000 (43966)
[2024-11-07 14:03:38 +0100] [43966] [INFO] Using worker: sync
[2024-11-07 14:03:38 +0100] [43967] [INFO] Booting worker with pid: 43967
[2024-11-07 14:03:38 +0100] [43968] [INFO] Booting worker with pid: 43968
[2024-11-07 14:03:39 +0100] [43969] [INFO] Booting worker with pid: 43969
[2024-11-07 14:03:39 +0100] [43970] [INFO] Booting worker with pid: 43970
^C
[2024-11-07 14:04:05 +0100] [43966] [INFO] Handling signal: int
[2024-11-07 14:04:05 +0100] [43970] [INFO] Worker exiting (pid: 43970)
[2024-11-07 14:04:05 +0100] [43969] [INFO] Worker exiting (pid: 43969)
[2024-11-07 14:04:05 +0100] [43967] [INFO] Worker exiting (pid: 43967)
[2024-11-07 14:04:05 +0100] [43968] [INFO] Worker exiting (pid: 43968)


In [18]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Kmodes - K=2 init: Huang"):
    # Selección y configuración de la técnica
    modelo_clusters = KModes(n_clusters=2, init='Huang', n_init=5)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Calcular métricas (ejemplo: Davies-Bouldin Index)

    # Necesidad de convertir a numericas las variables para el calculo de DBS: one hot encoding
    encoder = OneHotEncoder(sparse_output=False)
    X_encoded = encoder.fit_transform(df)

     #Calcular el Davies-Bouldin-Score
    davies_bouldin = davies_bouldin_score(X_encoded, labels)
    print(f"Davies-Bouldin Index: {davies_bouldin}")
    # Calcular las distancias entre todos los puntos utilizando distancia Hamming
    distancias = pairwise_distances( X_encoded, metric='hamming')

    # Calcular el Silhouette Score
    silhouette_avg = silhouette_score(distancias, labels, metric="precomputed")
    print(f"Silhouette Score promedio para clustering: {silhouette_avg}")
    
    # Registrar parámetros y métricas en MLflow
    mlflow.log_param("k", 2)
    mlflow.log_param("init_method", 'Huang')
    mlflow.log_metric("davies_bouldin_score", davies_bouldin )
    mlflow.log_metric("silhouette_score", silhouette_avg )
    

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K2-Huang")

    # Se finaliza el registro
    mlflow.end_run()

Davies-Bouldin Index: 2.7772839007585652
Silhouette Score promedio para clustering: 0.1944943204195812


2024/11/07 14:13:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Para contabilizar los elementos por cluster se utilizan las labels generadas

In [19]:
# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k2_huang = df.copy()
# Se agregan las labels generadas
df_kmeans_k2_huang['cluster'] = cluster_labels
df_kmeans_k2_huang['cluster'].value_counts()

cluster
0    5911
1    3622
Name: count, dtype: int64

Se prueban el resto de configuraciones de K-Modes

k=2 init ='Cao'

In [20]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Kmodes - K=2 init: Cao"):
    # Selección y configuración de la técnica
    modelo_clusters = KModes(n_clusters=2, init='Cao', n_init=5)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Calcular métricas (ejemplo: Davies-Bouldin Index)

    # Necesidad de convertir a numericas las variables para el calculo de DBS: one hot encoding
    encoder = OneHotEncoder(sparse_output=False)
    X_encoded = encoder.fit_transform(df)

     #Calcular el Davies-Bouldin-Score
    davies_bouldin = davies_bouldin_score(X_encoded, labels)
    print(f"Davies-Bouldin Index: {davies_bouldin}")
    # Calcular las distancias entre todos los puntos utilizando distancia Hamming
    distancias = pairwise_distances( X_encoded, metric='hamming')

    # Calcular el Silhouette Score
    silhouette_avg = silhouette_score(distancias, labels, metric="precomputed")
    print(f"Silhouette Score promedio para clustering: {silhouette_avg}")

    # Registrar parámetros y métricas en MLflow
    mlflow.log_param("k", 2)
    mlflow.log_param("init_method", 'Cao')
    mlflow.log_metric("davies_bouldin_score", davies_bouldin)
    mlflow.log_metric("silhouette_score", silhouette_avg )

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K2-Cao")

    # Se finaliza el registro
    mlflow.end_run()

Davies-Bouldin Index: 2.7772839007585652
Silhouette Score promedio para clustering: 0.1944943204195812


2024/11/07 14:17:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [21]:
# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k2_cao = df.copy()
# Se agregan las labels generadas
df_kmeans_k2_cao['cluster'] = cluster_labels
df_kmeans_k2_cao['cluster'].value_counts()

cluster
0    6346
1    3187
Name: count, dtype: int64

k=3 init= 'Huang'

In [22]:
with mlflow.start_run(experiment_id=exp_id, run_name="Kmodes - K=3 init: Huang"):
    # Selección y configuración de la técnica
    modelo_clusters = KModes(n_clusters=3, init='Huang', n_init=5)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_
    print(cluster_labels)
    # Calcular métricas (ejemplo: Davies-Bouldin Index)

    # Necesidad de convertir a numericas las variables para el calculo de DBS: one hot encoding
    encoder = OneHotEncoder(sparse_output=False)
    X_encoded = encoder.fit_transform(df)

     #Calcular el Davies-Bouldin-Score
    davies_bouldin = davies_bouldin_score(X_encoded, labels)
    print(f"Davies-Bouldin Index: {davies_bouldin}")

    # Calcular las distancias entre todos los puntos utilizando distancia Hamming
    distancias = pairwise_distances( X_encoded, metric='hamming')

    # Calcular el Silhouette Score
    silhouette_avg = silhouette_score(distancias, labels, metric="precomputed")
    print(f"Silhouette Score promedio para clustering: {silhouette_avg}")
    

    # Registrar parámetros y métricas en MLflow
    mlflow.log_param("k", 3)
    mlflow.log_param("init_method", 'Huang')
    mlflow.log_metric("davies_bouldin_score", davies_bouldin)
    mlflow.log_metric("silhouette_score", silhouette_avg )

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K3-Huang")

    # Se finaliza el registro
    mlflow.end_run()

[0 1 0 ... 0 1 1]
Davies-Bouldin Index: 2.7772839007585652
Silhouette Score promedio para clustering: 0.1944943204195812


2024/11/07 14:22:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [23]:
# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k3_huang = df.copy()
# Se agregan las labels generadas
df_kmeans_k3_huang['cluster'] = cluster_labels
df_kmeans_k3_huang['cluster'].value_counts()

cluster
0    4964
1    2850
2    1719
Name: count, dtype: int64

k=3 init=Cao

In [95]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import pairwise_distances

In [31]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Kmodes - K=3 init: Huang"):
    # Selección y configuración de la técnica
    modelo_clusters = KModes(n_clusters=3, init='Huang', n_init=5)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_
    print(cluster_labels)

    # Calcular métricas (ejemplo: Davies-Bouldin Index)

    # Necesidad de convertir a numericas las variables para el calculo de DBS: one hot encoding
    encoder = OneHotEncoder(sparse_output=False)
    X_encoded = encoder.fit_transform(df)

    #Calcular el Davies-Bouldin-Score
    davies_bouldin = davies_bouldin_score(X_encoded, cluster_labels)
    print(f"Davies-Bouldin Index: {davies_bouldin}")
   
   # Calcular las distancias entre todos los puntos utilizando distancia Hamming
    distancias = pairwise_distances( X_encoded, metric='hamming')

    # Calcular el Silhouette Score
    silhouette_avg = silhouette_score(distancias, cluster_labels, metric="precomputed")
    print(f"Silhouette Score promedio para clustering: {silhouette_avg}")


    # Registrar parámetros y métricas en MLflow
    mlflow.log_param("k", 3)
    mlflow.log_param("init_method", 'Cao')
    mlflow.log_metric("davies_bouldin_score", davies_bouldin)
    mlflow.log_metric("silhouette_score", silhouette_avg)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K3-Cao")

    # Se finaliza el registro
    mlflow.end_run()

    

[1 1 0 ... 0 1 1]
Davies-Bouldin Index: 3.5677670748626906
Silhouette Score promedio para clustering: 0.13158094307661888


2024/11/07 15:30:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [90]:
!%pip install scipy
!%pip install numpy


zsh:fg:1: no job control in this shell.
zsh:fg:1: no job control in this shell.


In [64]:
# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k3_cao = df.copy()
# Se agregan las labels generadas
df_kmeans_k3_cao['cluster'] = cluster_labels
df_kmeans_k3_cao['cluster'].value_counts()

cluster
0    4956
1    2870
2    1707
Name: count, dtype: int64

In [25]:
# Visualizar resultados en mlflow
!mlflow ui

[2024-11-07 14:26:20 +0100] [49269] [INFO] Starting gunicorn 23.0.0
[2024-11-07 14:26:20 +0100] [49269] [INFO] Listening at: http://127.0.0.1:5000 (49269)
[2024-11-07 14:26:20 +0100] [49269] [INFO] Using worker: sync
[2024-11-07 14:26:20 +0100] [49270] [INFO] Booting worker with pid: 49270
[2024-11-07 14:26:21 +0100] [49271] [INFO] Booting worker with pid: 49271
[2024-11-07 14:26:21 +0100] [49272] [INFO] Booting worker with pid: 49272
[2024-11-07 14:26:21 +0100] [49273] [INFO] Booting worker with pid: 49273
^C
[2024-11-07 14:59:40 +0100] [49269] [INFO] Handling signal: int
[2024-11-07 14:59:40 +0100] [49273] [INFO] Worker exiting (pid: 49273)
[2024-11-07 14:59:40 +0100] [49270] [INFO] Worker exiting (pid: 49270)
[2024-11-07 14:59:40 +0100] [49271] [INFO] Worker exiting (pid: 49271)
[2024-11-07 14:59:40 +0100] [49272] [INFO] Worker exiting (pid: 49272)


In [44]:
!%pip install seaborn

zsh:fg:1: no job control in this shell.


DBSCAN

------------------ EJEMPLOS Y PRUEBAS------------

Se pasa a probar otro método (clustering jerárquico)

In [9]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="AGClustering - K=3"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=3)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 3)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "AGC_K3")

    # Se finaliza el registro
    mlflow.end_run()

2024/05/08 21:36:20 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


Para contabilizar los elementos por cluster se utilizan las labels generadas

In [10]:
# Se preserva el conjunto de datos original para evitar problemas
df_agc_k3 = df.copy()
# Se agregan las labels generadas
df_agc_k3['cluster'] = cluster_labels
df_agc_k3['cluster'].value_counts()

cluster
1    4697
2    3817
0     371
Name: count, dtype: int64

Finalmente, se prueba un método que no tiene parámetros

In [11]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="MeanShift"):
    # Selección y configuración de la técnica
    modelo_clusters = MeanShift()
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # No hay parámetros por registrar

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "MS")

    # Se finaliza el registro
    mlflow.end_run()

Para contabilizar los elementos por cluster se utilizan las labels generadas

In [12]:
# Se preserva el conjunto de datos original para evitar problemas
df_ms = df.copy()
# Se agregan las labels generadas
df_ms['cluster'] = cluster_labels
df_ms['cluster'].value_counts()

cluster
0    8519
1     306
2      39
3      11
4       6
5       3
6       1
Name: count, dtype: int64

----

Prueba Nro. 2

- Kmeans


In [13]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="Kmeans - K=3"):
    # Selección y configuración de la técnica
    modelo_clusters = KMeans(n_clusters=3)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 3)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "Kmeans_K3")

    # Se finaliza el registro
    mlflow.end_run()

In [14]:
# Se preserva el conjunto de datos original para evitar problemas
df_kmeans_k3 = df.copy()
# Se agregan las labels generadas
df_kmeans_k3['cluster'] = cluster_labels
df_kmeans_k3['cluster'].value_counts()

cluster
0    4703
1    3876
2     306
Name: count, dtype: int64

In [15]:
df_kmeans_k3.to_csv("../../../data/final/resultados_kmeans_k3.csv", sep=";", index=False)

- Clustering Jerárquico

In [16]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id, run_name="AGClustering - K=2"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=2)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 2)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "AGC_K2")

    # Se finaliza el registro
    mlflow.end_run()

2024/05/08 21:40:54 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [17]:
# Se preserva el conjunto de datos original para evitar problemas
df_agc_k2 = df.copy()
# Se agregan las labels generadas
df_agc_k2['cluster'] = cluster_labels
df_agc_k2['cluster'].value_counts()

cluster
0    8514
1     371
Name: count, dtype: int64

In [21]:
df_agc_k2.to_csv("../../../data/final/resultados_df_agc_k2.csv", sep=";", index=False)

----

Experimentación #2

In [19]:
# Se genera el experimento en mlflow
exp_name_2 = 'Experimentación clustering_ap2 #2'
exp_id_2 = mlflow.create_experiment(name=exp_name_2)

Traceback (most recent call last):
  File "/Users/lorenadlmd/Desktop/Master BIGDATA/BIGDATA/Metodologia de proyectos/P1/13MBID-Proyectos/.env/lib/python3.12/site-packages/mlflow/store/tracking/file_store.py", line 302, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/lorenadlmd/Desktop/Master BIGDATA/BIGDATA/Metodologia de proyectos/P1/13MBID-Proyectos/.env/lib/python3.12/site-packages/mlflow/store/tracking/file_store.py", line 395, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/lorenadlmd/Desktop/Master BIGDATA/BIGDATA/Metodologia de proyectos/P1/13MBID-Proyectos/.env/lib/python3.12/site-packages/mlflow/store/tracking/file_store.py", line 1320, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^

In [20]:
# Por cada método se debe registrar la ejecución
with mlflow.start_run(experiment_id=exp_id_2, run_name="AGClustering - K=3"):
    # Selección y configuración de la técnica
    modelo_clusters = AgglomerativeClustering(n_clusters=3)
    
    trained_model = modelo_clusters.fit(df)
    cluster_labels = trained_model.labels_

    # Se realizar el cálculo de las métricas seleccionadas
    score_s = silhouette_score(df, cluster_labels)
    score_db = davies_bouldin_score(df, cluster_labels)
    score_ch = calinski_harabasz_score(df, cluster_labels)

    # Se registra el parámetro K
    mlflow.log_param('Valor K', 3)

    # Se registran las métricas de evaluación
    mlflow.log_metric('silhouette_score', score_s)
    mlflow.log_metric('davies_bouldin_score', score_db)
    mlflow.log_metric('calinski_harabasz_score', score_ch)

    # Se guarda el modelo generado
    mlflow.sklearn.log_model(trained_model, "AGC_K3")

    # Se finaliza el registro
    mlflow.end_run()

2024/05/08 21:41:20 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
